In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
  W_maxnorm = 3
  DROPOUT = {{choice([0.3,0.5,0.7])}}

  model = Sequential()
  model.add(Conv2D(64, (1, 5), padding='same', input_shape=(20, 1, 20),activation='leaky_relu',kernel_constraint=maxnorm(W_maxnorm)))
  model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
  model.add(Flatten())
  model.add(Dense(16,activation='leaky_relu',kernel_constraint=maxnorm(W_maxnorm)))
  model.add(Dropout(DROPOUT))
  model.add(Dense(1))

  myoptimizer = RMSprop(lr={{choice([0.01,0.001,0.0001,1e-5,1e-1])}}, rho=0.9, epsilon=1e-06)
  mylossfunc='mean_squared_error'
  model.compile(loss=mylossfunc, optimizer=myoptimizer)

  result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

  val_loss = np.amax(result.history['val_loss'])

  return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [10]:
def data():
  data_train = h5py.File('./data/Hold out Top 4%/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/Test set Regression/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [11]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_64x1_16',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [01:24<00:00,  8.45s/it, best loss: 0.26785823702812195]


In [12]:
X_train, Y_train, X_test, Y_test = data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True)

Epoch 1/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2227 - val_loss: 0.2254
Epoch 2/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2189 - val_loss: 0.2240
Epoch 3/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2157 - val_loss: 0.2262
Epoch 4/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2129 - val_loss: 0.2264
Epoch 5/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2110 - val_loss: 0.2263
Epoch 6/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2088 - val_loss: 0.2262
Epoch 7/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2077 - val_loss: 0.2285
Epoch 8/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2059 - val_loss: 0.2279
Epoch 9/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2050 - val_loss: 0.2298
Epoch 10/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2033 - val_loss: 0.2284

In [13]:
best_model.evaluate(X_test, Y_test)

710/710 [==============================] - 1s 2ms/step - loss: 0.3535


0.3535346984863281

In [14]:
best_model.save('../weights/regression/Hold out Top 4%/seq_64x1_16')

INFO:tensorflow:Assets written to: ../weights/regression/Hold out Top 4%/seq_64x1_16/assets
